In [1]:
import torch
import torchvision
from torchvision import transforms
import torch.nn.functional as F
from torch.nn import Conv2d, Linear

In [2]:
class CNN(torch.nn.Module):
    def __init__(self, hidden_channels, activationCNN, activationOut):
        super(CNN, self).__init__()
        torch.manual_seed(42)

        # in_channels=1 bo obraz jest jednokanałowy(greyscale)
        self.conv1 = Conv2d(1, hidden_channels, kernel_size=3, padding=1) # kernel 3x3, padding=1 pixel czyli output ma ten sam size czyli 28x28
        self.conv2 = Conv2d(hidden_channels, hidden_channels * 2, kernel_size=3, padding=1)
        self.fullyConnected = Linear(hidden_channels * 2 * 7 * 7, 10) # (hidden channels * 2 * 2) to  wyjscie z warstwy 2 konwolucyjnej a 7x7 to dlatego ze poczatkowy rozmiar obrazka jest 28x28
        # alepo każdej warstwie konwolucyjnej był pooling 2x2, który redukuje wektor wejściowy, czyli 28:(2x2) = 7
        self.activationCNN = activationCNN
        self.activationOut = activationOut

    def forward(self, x):
        x = self.conv1(x)
        x = self.activationCNN(x)
        x = F.max_pool2d(x, 2)

        x = self.conv2(x)
        x = self.activationCNN(x)
        x = F.max_pool2d(x, 2)

        x = x.view(x.size(0), -1)

        if self.activationOut == torch.softmax:
            x = self.activationOut(self.fullyConnected(x), dim=1)
        else:
            x = self.activationOut(self.fullyConnected(x))
        return x

In [3]:
model = CNN(hidden_channels=64, activationCNN=F.leaky_relu, activationOut=torch.sigmoid)
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [4]:
# Access the weights of each layer
conv1_weights = model.conv1.weight.clone()
conv2_weights = model.conv2.weight.clone()

In [5]:
conv1_weights[:5]

tensor([[[[ 0.2548,  0.2767, -0.0781],
          [ 0.3062, -0.0730,  0.0673],
          [-0.1623,  0.1958,  0.2938]]],


        [[[-0.2445,  0.2897,  0.0624],
          [ 0.2463,  0.0451,  0.1607],
          [-0.0471,  0.2570,  0.0493]]],


        [[[-0.1556,  0.0850, -0.1536],
          [-0.0391, -0.1354,  0.2211],
          [-0.2631, -0.1537, -0.0941]]],


        [[[-0.2004,  0.0315, -0.3292],
          [ 0.3010, -0.2832,  0.2573],
          [ 0.0555, -0.1082,  0.2060]]],


        [[[ 0.0520,  0.2693,  0.0364],
          [-0.1051,  0.0896, -0.0904],
          [ 0.1403,  0.2976,  0.1927]]]], grad_fn=<SliceBackward0>)

In [6]:
def train(model):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    return train_loss/len(train_loader.dataset)

In [7]:
def test(model):
    model.eval()
    correct = 0
    predictions = []
    targets = []
    inputs = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            predictions.append(pred)
            targets.append(target)
            inputs.append(data)
            correct += pred.eq(target.view_as(pred)).sum().item()
    print(correct)
    print(len(test_loader.dataset))
    accuracy = 100. * correct / len(test_loader.dataset)
    return accuracy, predictions, targets, inputs

In [8]:
# Funckja do trenowania modelu przez zadaną liczbę epok
def final_train(epochs, model):
    losses = []
    for epoch in range(0, epochs):
        loss = train(model)
        losses.append(loss)
    return losses

In [9]:
train_dataset = torchvision.datasets.MNIST('./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST('./data', train=False, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [11]:
losses = final_train(10, model)

In [12]:
acc, predictions, targets, inputs = test(model)

9884
10000


In [44]:
model.conv1.weight[0]

tensor([[[ 0.3455,  0.3502, -0.1280],
         [ 0.3648,  0.0399,  0.1467],
         [-0.1729,  0.2789,  0.4004]]], grad_fn=<SelectBackward0>)

In [45]:
conv1_weights[0]

tensor([[[ 0.2548,  0.2767, -0.0781],
         [ 0.3062, -0.0730,  0.0673],
         [-0.1623,  0.1958,  0.2938]]], grad_fn=<SelectBackward0>)

In [13]:
conv1_weights_after_training = model.conv1.weight
conv2_weights_after_training = model.conv2.weight

<font size="5">Można sobie powyswietlac wagi. Zwrócić uwagę, że named buffor jest pusty - po pruningu będzie tam maska pruningu.</font>

In [ ]:
print(list(model.conv1.named_parameters())) # nie wiem jeszcze czym sie roznia named_parameters od wag

In [ ]:
print(list(model.conv1.named_buffers()))

In [ ]:
print(model.conv1.weight)

<font size="5">Pruning wag konwolucyjnych. </font>

In [13]:
import torch.nn.utils.prune as prune

In [19]:
parameters_to_prune = (
    (model.conv1, 'weight'),  # Prune the weights of layer1
    (model.conv2, 'weight'),  # Prune the weights of layer2
)

In [20]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.2,  # Prune 20% of the parameters
)

In [21]:
losses = final_train(10, model)

In [22]:
acc, predictions, targets, inputs = test(model)

9897
10000


<font size="5">Wyswietlamy wagi itp, należy zwrócić uwagę, że named buffor się wypełnił maską 0 i 1. Po wywołaniu funkcji prune.remove wagi z maską=0 ustawiane są na 0.</font>

In [ ]:
module = model.conv1
print(list(module.named_parameters()))

In [ ]:
print(list(module.named_parameters()))

In [ ]:
print(list(module.named_buffers()))

In [ ]:
prune.l1_unstructured(module, name="bias", amount=3)

In [ ]:
print(list(module.named_parameters()))

In [ ]:
print(module.bias)

<font size="5">W wektorze weights waga, która po pruningu w named bufforze ma flagę 0 odrazu jest ustawiona na 0 natomiast w named_params dopiero po prune.remove.</font>

In [ ]:
print(module.weight)

In [ ]:
prune.remove(module, 'weight')

In [ ]:
prune.remove(model.conv2, 'weight')

In [ ]:
print(list(module.named_parameters()))

<font size="5">Ile procent usunietych wag</font>

In [ ]:
print("Sparsity in fc2.weight: {:.2f}%".format(100. * float(torch.sum(model.conv1.weight == 0))/ float(model.conv1.weight.nelement())))

In [ ]:
print("Sparsity in fc2.weight: {:.2f}%".format(100. * float(torch.sum(model.conv2.weight == 0))/ float(model.conv2.weight.nelement())))

In [23]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 20.00%


<font size="5">Pomysł na zrobienie lottery ticketa: <br>
- wytrenować model <br>
- zrobić pruning <br>
- przywrócić wagi <br>
- wywołać metodę prune.remove <br>
Jesli to nie zadziała to rzemieślniczo metoda, jechać element po elemencie po wadze i tam gdzie 0 w named bufforze albo w skopiowanym wektorze wag, tam ustawic 0.</font>

In [24]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.4,  # Prune 20% of the parameters
)

In [25]:
losses = final_train(10, model)

In [26]:
acc, predictions, targets, inputs = test(model)

9904
10000


In [27]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 52.00%


In [28]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.6,  
)

In [29]:
losses = final_train(10, model)

In [30]:
acc, predictions, targets, inputs = test(model)

9900
10000


In [31]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 80.80%


In [ ]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.6,  
)

In [ ]:
losses = final_train(1, model)

In [ ]:
acc, predictions, targets, inputs = test(model)

In [ ]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

In [32]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.8,  
)

In [33]:
losses = final_train(10, model)

In [34]:
acc, predictions, targets, inputs = test(model)

9888
10000


In [35]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 96.16%


In [ ]:
prune.remove(model.conv1, 'weight')
prune.remove(model.conv2, 'weight')

In [ ]:
losses = final_train(10, model)

In [ ]:
acc, predictions, targets, inputs = test(model)

In [13]:
import torch.nn.utils.prune as prune

In [14]:
parameters_to_prune = (
    (model.conv1, 'weight'),  # Prune the weights of layer1
    (model.conv2, 'weight'),  # Prune the weights of layer2
)

In [15]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.9996,  
)

In [16]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 99.96%


In [17]:
losses = final_train(10, model)

In [18]:
acc, predictions, targets, inputs = test(model)

1135
10000


In [19]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.9996,  
)

In [20]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 100.00%


<font size="5">Winning tickety</font>

In [46]:
import torch.nn.utils.prune as prune

In [47]:
parameters_to_prune = (
    (model.conv1, 'weight'),  # Prune the weights of layer1
    (model.conv2, 'weight'),  # Prune the weights of layer2
)

In [48]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.2,  # Prune 20% of the parameters
)

In [49]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 20.00%


<font size="3">Przepisanie wag</font>

In [50]:
model.conv1.weight[0]

tensor([[[ 0.3455,  0.3502, -0.1280],
         [ 0.3648,  0.0399,  0.1467],
         [-0.1729,  0.2789,  0.4004]]], grad_fn=<SelectBackward0>)

In [51]:
conv1_weights[0]

tensor([[[ 0.2548,  0.2767, -0.0781],
         [ 0.3062, -0.0730,  0.0673],
         [-0.1623,  0.1958,  0.2938]]], grad_fn=<SelectBackward0>)

In [25]:
model.conv1.weight[0][0][0]

tensor([ 0.3455,  0.3502, -0.1280], grad_fn=<SelectBackward0>)

In [26]:
model.conv1.weight[0][0][0][0]

tensor(0.3455, grad_fn=<SelectBackward0>)

In [28]:
for row in model.conv1.weight[0][0]:
    for weight in row:
        print(weight)

tensor(0.3455, grad_fn=<UnbindBackward0>)
tensor(0.3502, grad_fn=<UnbindBackward0>)
tensor(-0.1280, grad_fn=<UnbindBackward0>)
tensor(0.3648, grad_fn=<UnbindBackward0>)
tensor(0.0399, grad_fn=<UnbindBackward0>)
tensor(0.1467, grad_fn=<UnbindBackward0>)
tensor(-0.1729, grad_fn=<UnbindBackward0>)
tensor(0.2789, grad_fn=<UnbindBackward0>)
tensor(0.4004, grad_fn=<UnbindBackward0>)


In [52]:
len(model.conv1.weight)

64

In [53]:
len(model.conv1.weight[0])

1

In [54]:
len(model.conv1.weight[0][0])

3

In [134]:
def recoverOriginalWeightsW1():
    clone = model.conv1.weight.clone()
    for i, neuron in enumerate(model.conv1.weight):
        for j, row in enumerate(neuron[0]) :
            for k, weight in enumerate(row):
#                 print(clone[i][0][j][k])
#                 print(conv1_weights[i][0][j][k])
                if weight != 0: # pomijamy sprunowane wagi
                    clone[i][0][j][k].fill_(conv1_weights[i][0][j][k].detach())
    return clone                    

In [104]:
def printW(tensor):
    for i, neuron in enumerate(tensor):
        for j, row in enumerate(neuron[0]) :
            for k, weight in enumerate(row):
                print(tensor[i][0][j][k])
                print(conv1_weights[i][0][j][k])
                print(model.conv1.weight[i][0][j][k])

In [ ]:
clone = recoverOriginalWeightsW1()

In [105]:
model.conv1.weight = clone

In [120]:
def recoverOriginalWeights(original_tensor, replace_tensor):
    clone = original_tensor.clone()
    for i, neuron in enumerate(original_tensor):
        for j, row in enumerate(neuron[0]) :
            for k, weight in enumerate(row):
                if weight != 0: # pomijamy sprunowane wagi
                    clone[i][0][j][k].fill_(replace_tensor[i][0][j][k].detach())
    return clone                    

In [108]:
def printW(tensor):
    for i, neuron in enumerate(tensor):
        for j, row in enumerate(neuron[0]) :
            for k, weight in enumerate(row):
                print(tensor[i][0][j][k])
                print(conv2_weights[i][0][j][k])
                print(model.conv2.weight[i][0][j][k])

In [121]:
clone = recoverOriginalWeights(model.conv2.weight, conv2_weights)

In [123]:
model.conv2.weight = clone

In [ ]:
printW(clone)

In [125]:
losses = final_train(10, model)

In [126]:
acc, predictions, targets, inputs = test(model)

9889
10000


In [127]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 20.00%


In [131]:
parameters_to_prune = (
    (model.conv1, 'weight'),  # Prune the weights of layer1
    (model.conv2, 'weight'),  # Prune the weights of layer2
)

In [132]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.2,  # Prune 20% of the parameters
)

In [133]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 48.80%


In [145]:
clone = recoverOriginalWeightsW1()

In [146]:
model.conv1.weight = clone

In [147]:
clone = recoverOriginalWeights(model.conv2.weight, conv2_weights)

In [148]:
model.conv2.weight = clone

In [140]:
losses = final_train(10, model)

In [141]:
acc, predictions, targets, inputs = test(model)

9896
10000


In [142]:
parameters_to_prune = (
    (model.conv1, 'weight'),  # Prune the weights of layer1
    (model.conv2, 'weight'),  # Prune the weights of layer2
)

In [143]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.6,  # Prune 20% of the parameters
)

In [144]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 79.52%


In [ ]:
clone = recoverOriginalWeightsW1()
model.conv1.weight = clone
clone = recoverOriginalWeights(model.conv2.weight, conv2_weights)
model.conv2.weight = clone

In [149]:
losses = final_train(10, model)

In [150]:
acc, predictions, targets, inputs = test(model)

9905
10000


In [151]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 79.52%


In [152]:
parameters_to_prune = (
    (model.conv1, 'weight'),  # Prune the weights of layer1
    (model.conv2, 'weight'),  # Prune the weights of layer2
)

In [153]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.8,  # Prune 20% of the parameters
)

In [154]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 95.90%


In [155]:
clone = recoverOriginalWeightsW1()
model.conv1.weight = clone
clone = recoverOriginalWeights(model.conv2.weight, conv2_weights)
model.conv2.weight = clone

In [156]:
losses = final_train(10, model)

In [157]:
acc, predictions, targets, inputs = test(model)

9883
10000


<font size="5">Reinicjalizacja wag</font>

In [10]:
losses = final_train(10, model)

In [11]:
acc, predictions, targets, inputs = test(model)

9884
10000


In [12]:
import torch.nn.utils.prune as prune

In [13]:
parameters_to_prune = (
    (model.conv1, 'weight'),  # Prune the weights of layer1
    (model.conv2, 'weight'),  # Prune the weights of layer2
)

In [14]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.2,  # Prune 20% of the parameters
)

In [23]:
def randomReinitalizeW1(original_tensor):
    clone = original_tensor.clone()
    for i, neuron in enumerate(original_tensor):
        for j, row in enumerate(neuron[0]) :
            for k, weight in enumerate(row):
                if weight != 0: # pomijamy sprunowane wagi
                #if no specific weight initialization method is specified, the weights are initialized using a uniform distribution within the range (-0.1, 0.1)
                    tensor = torch.empty(1)
                    tensor.uniform_(-0.1, 0.1)
                    clone[i][0][j][k].fill_(tensor.item())
    return clone                    

In [24]:
model.conv1.weight = randomReinitalizeW1(model.conv1.weight)

In [25]:
model.conv2.weight = randomReinitalizeW1(model.conv2.weight)

In [26]:
losses = final_train(10, model)

In [27]:
acc, predictions, targets, inputs = test(model)

9895
10000


In [28]:
parameters_to_prune = (
    (model.conv1, 'weight'),  # Prune the weights of layer1
    (model.conv2, 'weight'),  # Prune the weights of layer2
)

In [29]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.4,  # Prune 20% of the parameters
)

In [30]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 52.00%


In [31]:
model.conv1.weight = randomReinitalizeW1(model.conv1.weight)

In [32]:
model.conv2.weight = randomReinitalizeW1(model.conv2.weight)

In [33]:
losses = final_train(10, model)

In [34]:
acc, predictions, targets, inputs = test(model)

9898
10000


In [35]:
parameters_to_prune = (
    (model.conv1, 'weight'),  # Prune the weights of layer1
    (model.conv2, 'weight'),  # Prune the weights of layer2
)

In [36]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.6,  # Prune 20% of the parameters
)

In [38]:
model.conv1.weight = randomReinitalizeW1(model.conv1.weight)

In [39]:
model.conv2.weight = randomReinitalizeW1(model.conv2.weight)

In [40]:
losses = final_train(10, model)

In [41]:
acc, predictions, targets, inputs = test(model)

9903
10000


In [42]:
parameters_to_prune = (
    (model.conv1, 'weight'),  # Prune the weights of layer1
    (model.conv2, 'weight'),  # Prune the weights of layer2
)

In [43]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.8,  # Prune 20% of the parameters
)

In [45]:
model.conv1.weight = randomReinitalizeW1(model.conv1.weight)

In [46]:
model.conv2.weight = randomReinitalizeW1(model.conv2.weight)

In [47]:
losses = final_train(10, model)

In [48]:
acc, predictions, targets, inputs = test(model)

9888
10000


In [49]:
parameters_to_prune = (
    (model.conv1, 'weight'),  # Prune the weights of layer1
    (model.conv2, 'weight'),  # Prune the weights of layer2
)

In [50]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.99,  # Prune 20% of the parameters
)

In [51]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
        )
    )
)

Global sparsity: 99.96%


In [52]:
model.conv1.weight = randomReinitalizeW1(model.conv1.weight)

In [53]:
model.conv2.weight = randomReinitalizeW1(model.conv2.weight)

In [54]:
losses = final_train(10, model)

In [55]:
acc, predictions, targets, inputs = test(model)

4238
10000
